# Composite Selectors

Combining selectors is one of the most powerful features of `soupsavvy`. By allowing you to mix and match different selectors, `soupsavvy` offers a highly flexible and customizable way to refine your search criteria. In this demo, we'll explore **Higher Order Selectors** and demonstrate how they can be leveraged to perform complex searches effectively.

**Higher Order Selectors** enable you to combine multiple selectors into a single, composite selector, enhancing your ability to target specific elements. Whether you're matching tags, attributes, or text content, these selectors can be used together to create more advanced search logic.

## Combinators

We'll begin with **Combinators**, which are inspired by the CSS concept of combinators. These allow you to pass multiple selectors and perform a search on a `Tag` object, mimicking the logic of CSS combinators but with the added power and flexibility of `soupsavvy`.

For more information on CSS combinators, you can refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Child_combinator).

While `BeautifulSoup` provides methods like `select` and `select_one` to find tags using CSS selectors, these are often limited by the constraints of vanilla CSS. `soupsavvy` goes beyond these limitations, offering more complex conditions such as regular expressions and other advanced selection logic that can be seamlessly combined using these powerful selectors.

### DescendantCombinator

The **Descendant Combinator** is one of the simplest and most frequently used combinators in CSS. It selects elements that match a second selector only if they have an ancestor that matches the first selector. In CSS, this relationship is represented by a single space `" "` between two selectors. For example, the following CSS:

```css
.book .price
```

matches all tags with the class `price` that are descendants of tags with the class `book`. For more details on CSS combinators, refer to the [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Descendant_combinator).

#### Usage

In `soupsavvy`, this logic is encapsulated in the `DescendantCombinator` class, which functions similarly to its CSS counterpart. The `DescendantCombinator` accepts two or more selectors and returns `Tag` objects that satisfy the descendant relationship between them.

Here's an example equivalent to the CSS above:

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, DescendantCombinator

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="price_section">
                <p class="price">Price: $20</p>
            </span>
        </div>
    """,
    features="lxml",
)

book_selector = AttributeSelector("class", value="book")
price_selector = AttributeSelector("class", value="price")
selector = DescendantCombinator(book_selector, price_selector)
selector.find(soup)

#### Creating with operator

As mentioned in previously, `soupsavvy` provides alternative way of creating some composite selectors by using operators. More concise way to create `DescendantCombinator` is by using the `>>` operator, which acts as syntactic sugar:

```python
DescendantCombinator(left, right) == left >> right
```

Where left selector matches ancestor and right selector matches descendant.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="price_section">
                <p class="price">Price: $20</p>
            </span>
        </div>
    """,
    features="lxml",
)

book_selector = AttributeSelector("class", value="book")
price_selector = AttributeSelector("class", value="price")
selector = book_selector >> price_selector
selector.find(soup)

#### Multiple selectors

`DescendantCombinator` allows you to chain together any number of selectors as positional arguments. When more than two selectors are provided, they are chained them in the order they appear, creating a more complex selection logic. This is similar to chaining selectors in CSS.

For instance, the following CSS:

```css
#available .book .price
```

matches all elements with the class `price` that are descendants of a `<div>` with the class `book`, which in turn is a descendant of a `<div>` with the ID `available`.

In `soupsavvy`, you can achieve this with `DescendantCombinator` as shown below:

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, IdSelector, TypeSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div class="book">
            <span class="title">Animal Farm</span>
            <p class="price">Price: $10</p>
        </div>
        <div id="available">
            <div class="book">
                <span class="title">Animal Farm</span>
                <span class="price_section">
                    <p class="price">Price: $20</p>
                </span>
            </div>
        </div>
    """,
    features="lxml",
)

available_selector = IdSelector("available")
book_selector = ClassSelector("book")
price_selector = ClassSelector("price")
selector = available_selector >> book_selector >> price_selector
selector.find(soup)

### Combinators equality

In `soupsavvy`, two combinators are considered equal only if they are of the same type and involve the exact same selectors in the same order. The order of selectors is significant, meaning `left >> right` is not equal to `right >> left`.

In [ ]:
from soupsavvy import AttributeSelector, DescendantCombinator

book_selector = AttributeSelector("class", value="book")
price_selector = AttributeSelector("class", value="price")

print(
    "left >> right == DescendantCombinator(left, right):",
    book_selector >> price_selector
    == DescendantCombinator(book_selector, price_selector),
)
print(
    "left >> right == right >> left:",
    price_selector >> book_selector == book_selector >> price_selector,
)

### Not Recursive search

Setting `recursive` parameter to `False` will return element only if element matched by first selector is a child of searched element. This logic is in place for all `soupsavvy` combinators.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <span class="not_child_book">
            <div class="book">
                <span class="title">Animal Farm</span>
                <span class="price_section">
                    <p class="price">Price: $50</p>
                </span>
            </div>
        </span>
        <div class="book">
            <span class="title">Animal Farm</span>
            <span class="price_section">
                <p class="price">Price: $20</p>
            </span>
        </div>
    """,
    features="html.parser",
)

book_selector = AttributeSelector("class", value="book")
price_selector = AttributeSelector("class", value="price")
selector = book_selector >> price_selector
selector.find(soup, recursive=False)

In this case `Price: $50` is not selected, as it's ancestor `<div class="book">` that matched first selector is not a direct parent of searched element.

### ChildCombinator

`ChildCombinator` mirrors the behavior of the CSS child combinator. In CSS, a child combinator (`>` symbol) selects elements that are direct children of a specified parent element. This is a stricter relationship than the descendant combinator, as it only matches elements that are immediate children of a given element, not just any descendant.

For example, in CSS:

```css
div > .price
```

This selector matches all elements with the class `price` that are direct children of `<div>` element. For more details refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Child_combinator).

#### Usage

In `soupsavvy`, this logic is implemented with the `ChildCombinator` class. It accepts two or more selectors as positional arguments and returns elements that match the specified child-parent relationships.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, ChildCombinator, TypeSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div>
            <span class="title">Animal Farm</span>
            <span class="discount">
                <h2>Discounted</h2>
                <p class="price">Price: $15</p>
            </span>
            <p class="price">Price: $20</p>
        </div>
    """,
    features="lxml",
)

book_selector = TypeSelector("div")
price_selector = AttributeSelector("class", value="price")
selector = ChildCombinator(book_selector, price_selector)
selector.find(soup)

#### Creating with operator

More concise way to create `ChildCombinator` is by using the `>` operator, which is consistent with CSS syntax:

```python
ChildCombinator(left, right) == left > right
```

Where left selector matches parent and right selector matches child.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, TypeSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div>
            <span class="title">Animal Farm</span>
            <span class="discount">
                <h2>Discounted</h2>
                <p class="price">Price: $15</p>
            </span>
            <p class="price">Price: $20</p>
        </div>
    """,
    features="lxml",
)

book_selector = TypeSelector("div")
price_selector = ClassSelector("price")
selector = book_selector > price_selector
selector.find(soup)

### Combining Combinators

Just like any other `soupsavvy` selector, combinators can be used as inputs to other higher-order selectors. For instance, you can define a combination of parent-child and descendant relationships within a single selector. In CSS, this would be represented as:

```css
#available > div .price
```

In `soupsavvy`, this logic can be replicated by using both the `ChildCombinator` and the `DescendantCombinator` together. This combination allows you to specify that tag with id `available` should contain a direct child `<div>`, which in turn should contain a descendant element with the class `price`.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, IdSelector, TypeSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $30</p>
        <div>
            <span class="title">Animal Farm</span>
            <p class="price">Price: $10</p>
        </div>
        <div id="available">
            <div>
                <span class="title">Animal Farm</span>
                <span class="discount">
                    <h2>Discounted</h2>
                    <p class="price">Price: $15</p>
                </span>
                <p class="price">Price: $20</p>
            </div>
        </div>
    """,
    features="lxml",
)

available_selector = IdSelector("available")
book_selector = TypeSelector("div")
price_selector = ClassSelector("price")
selector = (available_selector > book_selector) >> price_selector
selector.find(soup)

#### Caveats

When combining selectors with operators in `soupsavvy`, it's important to understand that some operators have higher precedence than others, which can affect the order in which expressions are evaluated.

For example, the `right shift` operator (`>>`) has higher precedence than the `greater than` operator (`>`). This means that in an expression like:

```python
left > middle >> right
```

The `>>` operation is performed first, resulting in:

```python
ChildCombinator(left, DescendantCombinator(middle, right))
```

In contrast, if you add parentheses to explicitly define the order of operations:

```python
(left > middle) >> right
```

This expression would be interpreted as:

```python
DescendantCombinator(ChildCombinator(left, middle), right)
```

#### Example

In provided example, the order of selectors isn't significant because both the `ChildCombinator` and `DescendantCombinator` are commutative — meaning the order in which you apply them doesn't affect the final result. However, this may not always be the case, so to avoid ambiguity and ensure expected behavior, it's always safer to include parentheses when combining operators.

In [ ]:
from soupsavvy import (
    ChildCombinator,
    ClassSelector,
    DescendantCombinator,
    IdSelector,
    TypeSelector,
)

available_selector = IdSelector("available")
book_selector = TypeSelector("div")
price_selector = ClassSelector("price")

print(
    DescendantCombinator(
        ChildCombinator(available_selector, book_selector), price_selector
    )
    == (available_selector > book_selector) >> price_selector
)

### NextSiblingCombinator

`NextSiblingCombinator` replicates the behavior of the CSS adjacent sibling combinator. In CSS, the next (aka adjacent) sibling combinator (denoted by the `+` symbol) selects an element that directly follows a specified sibling element.

For example, in CSS:

```css
div + .price
```

This selector matches all elements with the class `price` that are the immediate next siblings of `<div>` elements. For more details, refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Next-sibling_combinator).

#### Usage

In `soupsavvy`, the `NextSiblingCombinator` class implements this logic. It takes two or more selectors as positional arguments and returns elements that match the specified preceding-adjacent sibling relationships. Example below demonstrate different example by using `PatternSelector` to match text content, element with class `price` that is immediate sibling of element with text content `Discounted` will be selected.	  

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, NextSiblingCombinator, PatternSelector

soup = BeautifulSoup(
    """
        <h2>Discounted</h2>
        <span>Unavailable</span>
        <p class="price">Price: $10</p>
        <h1>Discounted</h1>
        <p class="price">Price: $15</p>
    """,
    features="lxml",
)

discount_selector = PatternSelector("Discounted")
price_selector = ClassSelector("price")
selector = NextSiblingCombinator(discount_selector, price_selector)
selector.find(soup)

#### Creating with operator

More concise way to create `NextSiblingCombinator` is by using the `+` operator, which is consistent with CSS syntax:

```python
NextSiblingCombinator(left, right) == left + right
```

Where left selector matches preceding sibling and right selector matches next sibling.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, PatternSelector

soup = BeautifulSoup(
    """
        <h2>Discounted</h2>
        <span>Unavailable</span>
        <p class="price">Price: $10</p>
        <h1>Discounted</h1>
        <p class="price">Price: $15</p>
    """,
    features="lxml",
)

discount_selector = PatternSelector("Discounted")
price_selector = ClassSelector("price")
selector = discount_selector + price_selector
selector.find(soup)

In case of having multiple selectors, they are chained in the order they appear, first selector matches element and each subsequent selector matches next sibling of previously matched element.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, PatternSelector

soup = BeautifulSoup(
    """
        <h2>Discounted</h2>
        <span>Breaking relationship :(</span>
        <p class="price">Price: $15</p>
        <span class="title">Animal Farm 1</span>
        
        <h2>Discounted</h2>
        <p class="price">Price: $15</p>
        <span>Breaking relationship :(</span>
        <span class="title">Animal Farm 2</span>
        
        <h2>Discounted</h2>
        <p class="price">Price: $15</p>
        <span class="title">Animal Farm 3</span>
    """,
    features="lxml",
)

discount_selector = PatternSelector("Discounted")
price_selector = ClassSelector("price")
title_selector = ClassSelector("title")
selector = discount_selector + price_selector + title_selector
selector.find(soup)

### Reducing combinators

When using multiple selectors for Higher Order Selector, they can be *reduced* into one by chaining them with operator, `functools.reduce` can be used for this purpose in a convenient way.

In [ ]:
from functools import reduce
from operator import add

from bs4 import BeautifulSoup

from soupsavvy import ClassSelector

soup = BeautifulSoup(
    """
       <span class="c1"></span>
       <span class="c2"></span>
       <span class="c3"></span>
       <span class="c4"></span>
       <span class="c5"></span>
    """,
    features="lxml",
)

selector = reduce(add, (ClassSelector(f"c{i}") for i in range(1, 6)))
selector.find(soup)

### SubsequentSiblingCombinator

`SubsequentSiblingCombinator` emulates the behavior of the CSS subsequent sibling combinator. In CSS, the subsequent sibling combinator (denoted by the `~` symbol) selects elements that follow the element matched by the first selector and share the same parent.

For example, in CSS:

```css
div ~ .price
```

This selector matches all elements with class `price` that are siblings of `<div>` and appear after it in document. For more details, refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Subsequent-sibling_combinator).

#### Usage

In `soupsavvy`, the `SubsequentSiblingCombinator` class provides similar functionality. It accepts two or more selectors as positional arguments and returns elements that match the specified preceding-subsequent sibling relationships. For more information on search logic when multiple selectors are provided, refer to the `NextSiblingCombinator` section.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, SubsequentSiblingCombinator, TypeSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $25</p>
        <h2>Discounted</h2>
        <span>Bargain!!!</span>
        <p class="price">Price: $15</p>
        <p class="price">Price: $10</p>
    """,
    features="lxml",
)

discount_selector = TypeSelector("h2")
price_selector = ClassSelector("price")
selector = SubsequentSiblingCombinator(discount_selector, price_selector)
selector.find_all(soup)

#### Creating with operator

More concise way to create `SubsequentSiblingCombinator` is by using the `*` operator:

```python
SubsequentSiblingCombinator(left, right) == left * right
```

Where left selector matches preceding sibling and right selector matches subsequent sibling.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, TypeSelector

soup = BeautifulSoup(
    """
        <p class="price">Price: $25</p>
        <h2>Discounted</h2>
        <span>Bargain!!!</span>
        <p class="price">Price: $15</p>
        <p class="price">Price: $10</p>
    """,
    features="lxml",
)

discount_selector = TypeSelector("h2")
price_selector = ClassSelector("price")
selector = discount_selector * price_selector
selector.find(soup)

## Logical Selectors

Another category of Higher Order Selectors in `soupsavvy` is **Logical Selectors**. These selectors enables you to create new selectors by combining multiple selectors using logical operators like `AND`, `OR`, `NOT` and `XOR`.

### AndSelector

`AndSelector` is the `soupsavvy` equivalent of the CSS compound selector. In CSS, a compound selector is a sequence of simple selectors that are not separated by a combinator. An element matches a compound selector if it satisfies all the simple selectors in the sequence. For example, the following CSS compound selector:

```css
p.price
```

matches all `<p>` elements with the class `price`. For more information about this selector refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_selectors/Selector_structure#compound_selector).

#### Usage

In `soupsavvy`, the `AndSelector` class achieves similar functionality. It accepts two or more selectors as positional arguments and returns elements that match **all** the specified selectors. This allows for highly specific and precise element selection.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AndSelector, ClassSelector, TypeSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <span class="price">Price: $30</p>
        <p class="price">Price: $20</p>
    """,
    features="lxml",
)

p_selector = TypeSelector("p")
price_selector = ClassSelector("price")
selector = AndSelector(p_selector, price_selector)
selector.find(soup)

#### Creating with operator

More concise way to create `AndSelector` is by using the `&` operator:

```python
AndSelector(left, right) == left & right
```

Where both left and right selectors must match for element to be selected.

It accepts two or more selectors as positional arguments and returns elements that match **all** of them.

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector, ClassSelector, PatternSelector, TypeSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="price">Price: $30</p>
        <p class="price">Price: $20</p>
        <p class="price" href="/shop">Price: €12</p>
        <span class="price" href="/shop">Price: $18</p>
        <p class="price" href="/shop">Price: $15</p>
    """,
    features="lxml",
)

selector = (
    TypeSelector("p")
    & ClassSelector("price")
    & AttributeSelector("href", value="/shop")
    & PatternSelector(re.compile(r"\$\d+"))
)
selector.find(soup)

### SelectorList

`SelectorList` is `soupsavvy`'s counterpart of the CSS selector list. In CSS, a selector list is a comma-separated collection of selectors, and an element matches the list if it satisfies any of the individual selectors. For example, consider the following CSS selector list:

```css
h1, h2
```

This selector list matches all `<h1>` and `<h2>` elements. For more information selector list refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/Selector_list).

#### Usage

In `soupsavvy`, `SelectorList` (also known as `OrSelector`, which is an alias) offers similar functionality. It accepts two or more selectors as positional arguments and returns elements that match **any** of the specified selectors.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, OrSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <span class="discount">
            <p class="price">Price: $25</p>
        </span>
        <p class="title">Brave New World</p>
        <p class="price">Price: $15</p>
    """,
    features="lxml",
)

title_selector = ClassSelector("title")
price_selector = ClassSelector("price")
discount_price_selector = title_selector + ClassSelector("discount") > price_selector
standard_price_selector = title_selector + price_selector

selector = OrSelector(discount_price_selector, standard_price_selector)
selector.find_all(soup)

#### Creating with operator

More concise way to create `SelectorList` is by using the `|` operator:

```python
SelectorList(left, right) == left | right
```
Where any of left or right selector must match for element to be selected.

It accepts two or more selectors as positional arguments and returns elements that match **any** of them.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <span class="discount">
            <p class="price">Price: $25</p>
        </span>
        <p class="title">Brave New World</p>
        <p class="price">Price: $15</p>
    """,
    features="lxml",
)

title_selector = ClassSelector("title")
price_selector = ClassSelector("price")
discount_price_selector = title_selector + ClassSelector("discount") > price_selector
standard_price_selector = title_selector + price_selector

selector = discount_price_selector | standard_price_selector
selector.find_all(soup)

#### Equality

The `SelectorList` in `soupsavvy` is unique in that it functions both as a combinator (due to its CSS-like use case) and as a logical selector (because of its general logic). Unlike other combinators, the order of selectors within a `SelectorList` is not significant, changing the order will not affect the results.

In [ ]:
from soupsavvy import ClassSelector, SelectorList

discount_selector = ClassSelector("discount")
price_selector = ClassSelector("price")

print(
    SelectorList(discount_selector, price_selector)
    == SelectorList(price_selector, discount_selector)
)

Additionally, two `SelectorList` instances can be considered equal even if they contain a different number of selectors, as long as they match the same elements.

In [ ]:
from soupsavvy import AttributeSelector, ClassSelector, SelectorList

discount_selector = ClassSelector("discount")
price_selector = ClassSelector("price")
another_price_selector = AttributeSelector("class", value="price")

print(
    SelectorList(discount_selector, price_selector)
    == SelectorList(discount_selector, price_selector, another_price_selector)
)

#### API shortcuts

`SelectorList` can also be created using `soupsavvy` API shortcut functions. These functions—`is_`, `or_`, and `where_`—provide alternative ways to instantiate a `SelectorList`. All these functions are equivalent and can be used interchangeably, depending on the context or the developer's preference.

For example, in CSS:

```css
:is(h1, h2)
:where(h1, h2)
```

Both `:is` and `:where` match all `<h1>` and `<h2>` elements. In `soupsavvy`, the `is_` and `where_` functions mimic these pseudo-classes. While they differ in terms of specificity in CSS, here, they are interchangeable. `or_` function serves as a more intuitive alias, especially when using `SelectorList` in a logical selector context.

For more information about these CSS pseudo-classes, refer to [:is()](https://developer.mozilla.org/en-US/docs/Web/CSS/:is) and [:where()](https://developer.mozilla.org/en-US/docs/Web/CSS/:where).

### NotSelector

`NotSelector` in `soupsavvy` corresponds to the CSS `:not()` pseudo-class. In CSS, `:not()` is used to exclude elements that match a certain selector. It allows you to apply styles to elements that do not match the specified criteria. For example:

```css
:not(.discount)
```

This selector matches all  elements except those that have the class `discount`. For further details, refer to [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/:not).

#### Usage

In `soupsavvy`, `NotSelector` provides similar functionality. It accepts one or more selectors as arguments and returns elements that do **not** match **any** of the specified selectors.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, NotSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = NotSelector(discount_selector)
selector.find(soup)

#### Creating with operator

More concise way to create `NotSelector` is by using the `~` operator:

```python
NotSelector(left) == ~selector
```
In this case, the element must not match the given selector to be selected.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = ~discount_selector
selector.find(soup)

#### Multiple selectors

It accepts two or more selectors as positional arguments and returns elements that do **not** match **any** of them.  Since the `~` operator accepts only one operand, you can use `SelectorList` to negate multiple selectors:

```python
NotSelector(left, right) == ~(left | right)
```

This is equivalent to the logical expression:

```python
if not any([left, right]):
    ...
```

In [ ]:

import re

from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, NotSelector, PatternSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
dollars_selector = PatternSelector(re.compile(r"\$\d+"))
selector = NotSelector(discount_selector, dollars_selector)
selector.find(soup)

Alternative way of defining `NotSelector` with multiple selectors is by using `~` operator on `SelectorList`:

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, PatternSelector

soup = BeautifulSoup(
    """
        <p class="price discount">Price: €10</p>
        <p class="price">Price: $20</p>
        <p class="price">Price: €15</p>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
dollars_selector = PatternSelector(re.compile(r"\$\d+"))
selector = ~(discount_selector | dollars_selector)
selector.find(soup)

### XORSelector

`XORSelector` is `soupsavvy`'s counterpart of the logical XOR operation, where an element is selected only if it matches exactly one of the provided selectors, but not more than one.

In CSS, there's no direct equivalent to `XORSelector`, but this can be achieved through selector list with compound selectors and `:not()` pseudo-class. For example this CSS:

```css 
span:not(.discount), .discount:not(span)
```

Matches all `<span>` elements that do not have the class `discount` and all elements with the class `discount` that are not `<span>` elements.

#### Usage

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, TypeSelector, XORSelector

soup = BeautifulSoup(
    """
        <span class="discount">Buy!</span>
        <p class="price">Price: $10</p>
        <span class="price">Price: $20</span>
        <p class="discount">Price: $30</p>
    """,
    features="lxml",
)

discount_selector = ClassSelector("discount")
span_selector = TypeSelector("span")
selector = XORSelector(discount_selector, span_selector)
selector.find_all(soup)

This logic could be achieved by combining `SelectorList` and `NotSelector` in `soupsavvy`.

In [ ]:
from soupsavvy import ClassSelector, TypeSelector

discount_selector = ClassSelector("discount")
span_selector = TypeSelector("span")

xor = (discount_selector & ~span_selector) | (~discount_selector & span_selector)

#### Multiple selectors

However, it is not scalable in case of having multiple selectors. `XORSelector` provides a more concise and readable way to express this logic. It accept two or more selectors as positional arguments and returns elements that match **exactly one** of the provided selectors.

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, PatternSelector, TypeSelector, XORSelector

soup = BeautifulSoup(
    """
        <span class="discount">Buy!</span>
        <p class="price">Price: $10</p>
        <span class="price">Price: $20</span>
        <p class="discount">Price: $30</p>
        <p class="discount">Only Today: $8</p>
        <span>Only Today: $7</span>
        <p>Only Today: $6</p>
    """,
    features="lxml",
)

discount_selector = ClassSelector("discount")
span_selector = TypeSelector("span")
only_today_selector = PatternSelector(re.compile("Only Today:"))
selector = XORSelector(discount_selector, span_selector, only_today_selector)
selector.find_all(soup)

## Relative Selectors

The next category of Higher Order Selectors in `soupsavvy` is **Relative Selectors**. As outlined by [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_selectors/Selector_structure#relative_selector), in CSS, a relative selector represents an element in relation to one or more anchor elements, typically introduced by a combinator. If a relative selector doesn't start with an explicit combinator, it is interpreted as having an implied descendant combinator.

### CSS vs soupsavvy

In CSS, relative selectors are generally used within specific contexts, such as the `:has()` pseudo-class. For example, consider this CSS:

```css
:has(> .price)
```

This selector matches any element that has a direct child with the class `price`. Here, the `anchor element` is the one being matched by the `> .price` relative selector. However, CSS restricts the use of relative selectors in a selector list, allowing them only in specific contexts.

In contrast, `soupsavvy` provides greater flexibility with relative selectors. These selectors can be used in any context, allowing you to search for elements relative to an `anchor element` passed directly to the `find` methods of these selectors.

### RelativeChild

Consider an example where the `RelativeChild` selector is used to find all elements with the class `price` that are direct children of the `body` element.

In this case:
- The `anchor element` is the `body` element.
- The `relative element` is any element with the class `price` that is a direct child of the `body` element.

In CSS, this would correspond to the selector `body > .price`, where `body` serves as the dynamically defined `anchor element`.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector
from soupsavvy.selectors.relative import RelativeChild

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $30</p>
    """,
    features="html.parser",
)

selector = AttributeSelector("class", value="price")
relative = RelativeChild(selector)
relative.find(soup)

### Anchor

You can create relative selectors in `soupsavvy` using the `Anchor` class, which allows you to chain selectors using specific operators:

- `>>` creates a `RelativeDescendant`
- `>` creates a `RelativeChild`
- `+` creates a `RelativeNextSibling`
- `*` creates a `RelativeSubsequentSibling`

These operators function similarly to how they are used with combinators, allowing you to define relationships between elements relative to a given anchor.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import Anchor, AttributeSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $30</p>
    """,
    features="html.parser",
)

selector = AttributeSelector("class", value="price")
relative = Anchor > selector
print(type(relative))
relative.find(soup)

### Behavior of Relative Selectors

Relative selectors in `soupsavvy` have unique behaviors that distinguish them from other selectors:

- **`RelativeChild`**: Always returns direct children of the searched element, regardless of the `recursive` parameter.
- **`RelativeDescendant`**: Always returns descendants of the searched element, even if `recursive` is set to `False`.
- **`RelativeNextSibling`** and **`RelativeSubsequentSibling`**: Always search for siblings of the searched element.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector
from soupsavvy.selectors.relative import RelativeChild

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $30</p>
        <div>
            <p class="price">Price: $40</p>
        </div>
    """,
    features="html.parser",
)

selector = AttributeSelector("class", value="price")
relative = RelativeChild(selector)

print(relative.find_all(soup, recursive=False))
print(relative.find_all(soup, recursive=True))

### RelativeNextSibling

Let's say you want to find elements that are immediate siblings of the `div` (searched element) and have the class `price`. You would use `RelativeNextSibling`:

This will return elements that are the immediate next sibling of the `div` element and have the class `price`. Since only one element can be the next sibling, this selector will return at most one element.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import AttributeSelector
from soupsavvy.selectors.relative import RelativeNextSibling

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <div class="section">Book 1</div>
        <p class="price">Price: $30</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $10</p>
    """,
    features="lxml",
)

selector = AttributeSelector("class", value="price")
relative = RelativeNextSibling(selector)
relative.find_all(soup.div)  # type: ignore

### Recursivity

Another powerful use of relative selectors in `soupsavvy` is their ability to "lock" the recursive behavior of other selectors. For example, when used within a `SelectorList`, `RelativeChild` ensures that matching elements are only found among the direct children of the searched element, regardless of the `recursive` parameter's setting. This allows for more precise control over the scope of element selection in complex queries.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import Anchor, AttributeSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <p class="price">Price: $30</p>
        <div>
            <p class="price">Price: $40</p>
            <a href="/shop">Buy Now</a>
        </div>
    """,
    features="html.parser",
)

price_selector = AttributeSelector("class", value="price")
shop_selector = AttributeSelector("href", value="/shop")
and_selector = (Anchor > price_selector) | shop_selector
and_selector.find_all(soup, recursive=True)

### HasSelector

The `HasSelector` in `soupsavvy` corresponds to the CSS `:has()` pseudo-class. As explained by [Mozilla](https://developer.mozilla.org/en-US/docs/Web/CSS/:has), this pseudo-class represents an element if any of the relative selectors passed as an argument match at least one element when anchored against this element.

#### CSS vs soupsavvy

In CSS, the `:has()` pseudo-class allows for matching an element based on the presence of its child or another relative element. For example:

```css
:has(> .price)
```

This CSS selector matches any element that has a direct child with the class `price`. Here, the element matched by the `:has()` pseudo-class is the one that contains the `.price` element as a direct child.

In `soupsavvy`, the `HasSelector` provides similar functionality. It accepts one or more selectors as arguments and returns elements if any of the relative selectors match at least one element when anchored against the current element.

#### Default Recursive Search

When you use the `HasSelector` without specifying a combinator, it defaults to the descendant combinator. This means that any selector passed to `HasSelector` that is not a `Relative Selector` or is `RelativeDescendant` will use the `recursive` search logic. In practical terms, this will search through all descendants of the anchor element to find matches. For instance, if a `<body>` element matches the selector, its parent `<html>` element will also be selected.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Brave New World</span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
        <span>
            <span class="discount">
                <p class="price">Price: $15</p>
            </span>
        </span>
        <p class="price">Price: $20</p>
    </div>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
selector = HasSelector(discount_selector)
selector.find_all(soup)

#### Multiple Selectors

When multiple selectors are used with `HasSelector`, it will return elements that match **any** of the provided selectors.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector, PatternSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Brave New World</span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
        <span class="discount">
            <p class="price">Price: $15</p>
        </span>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span>Bestseller</span>
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    """,
    features="html.parser",
)

discount_selector = ClassSelector("discount")
bestseller_selector = PatternSelector("Bestseller")
selector = HasSelector(discount_selector, bestseller_selector)
selector.find_all(soup)

#### Relative Selectors

By using relative selectors, you can define specific relationships other than descendants between the anchor and relative elements. For instance, you can search for elements that have a specific next sibling, rather than just a descendant.

To find elements that have a direct next sibling with the class `price`, you would use `RelativeNextSibling` within `HasSelector`:

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import Anchor, AttributeSelector, HasSelector

soup = BeautifulSoup(
    """
        <p class="title">Animal Farm</p>
        <p class="discount">Price: $20</p>
        <div class="section">Book 1</div>
        <p class="price">Price: $30</p>
    """,
    features="lxml",
)

selector = AttributeSelector("class", value="price")
has_selector = HasSelector(Anchor + selector)
has_selector.find(soup)

#### Compound Selectors

You can mimic CSS compound selectors with the `:has()` pseudo-class by combining `HasSelector` with `AndSelector`. For example:

```css
div:has(> .price)
```

The above CSS would match any `<div>` element that has a direct child with the class `price`.

In `soupsavvy`, this can be achieved by combining `HasSelector` with `RelativeChild` and `AndSelector`.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import Anchor, ClassSelector, HasSelector, TypeSelector

soup = BeautifulSoup(
    """
        <div><p class="discount">Price: $20</p></div>
        <span><p class="price">Price: $30</p></div>
        <div><p class="price">Price: $40</p></div>
    """,
    features="lxml",
)

relative = Anchor > ClassSelector("price")
has_selector = HasSelector(relative) & TypeSelector("div")
has_selector.find(soup)

#### Mixed Relative Selectors

You can also mix different relative selectors within `HasSelector`. For example, to find elements that have a next sibling with text content `Read more` or child with class `price`, you would use:

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import Anchor, AttributeSelector, HasSelector, PatternSelector

soup = BeautifulSoup(
    """
        <div class="newspaper">
            <p class="title">New York Times</p>
            <h1>Only 5$!</h1>
        </div>
        <p>Read more</p>
        <div class="book">
            <span>Animal Farm</span>
            <p class="price">Price: $30</p>
        </div>
    """,
    features="lxml",
)

relative_newspaper = Anchor + PatternSelector("Read more")
relative_book = Anchor > AttributeSelector("class", value="price")
has_selector = HasSelector(relative_newspaper, relative_book)
has_selector.find_all(soup)

#### RelativeSubsequentSibling

The `RelativeSubsequentSibling` within `HasSelector` can be used to find elements that are siblings preceding a specific **relative element**. For example, to find all siblings preceding the element with text content `Stop` you would use:

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import Anchor, HasSelector, PatternSelector

soup = BeautifulSoup(
    """
        <div class="countdown">4</div>
        <span>3</span>
        <p id="fgai23">2</p>
        <a href="https://example.com">1</a>
        <span>Stop</span>
        <p>Go</p>
    """,
    features="lxml",
)

relative_countdown = Anchor * PatternSelector("Stop")
has_selector = HasSelector(relative_countdown)
has_selector.find_all(soup)

## Nth Selectors

CSS provides several pseudo-classes such as `nth-child`, `nth-of-type`, and others, which allow you to select elements based on their position within a list of siblings. However, these CSS selectors are limited to selecting elements based on their ordinal position among all siblings. `soupsavvy` extends this functionality with a set of `Nth` selectors that allow you to select the nth element that matches a given selector, rather than just the nth child.

The `soupsavvy.selectors.nth` module introduces three components:

- **NthOfSelector**
- **NthLastOfSelector**
- **OnlyOfSelector**

Which allow you to perform more advanced selections based on the position of elements that match specific criteria.

#### CSS vs soupsavvy

For instance, if you want to select every 2nd element with a class `price`, this would be impossible in pure CSS, because:

```css
.price:nth-child(2n)
```

would select every element that is the 2nd child **and** has the class `price`, not every 3rd element with the class `price`!

In `soupsavvy`, this can be achieved with:

```python
NthOfSelector(ClassSelector('price'), nth="3n")
```

This selector would correctly select every 3rd element that matches the `price` class.

#### NthOfSelector

The `NthOfSelector` allows you to select elements that match a specified pattern of occurrence, based on the `nth` rule. Provided `nth` needs to be valid CSS nth rule `(<An+B> | even | odd)`. For example, to select every 2nd element with a class `price`, you would use:

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector
from soupsavvy.selectors.nth import NthOfSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <p class="price discount">Price: €1</p>
        <p class="price">Price: $2</p>
        <span>Bestseller</span>
        <p class="price">Price: €3</p>
        <p class="price">Price: €4</p>
        <p class="price">Price: €5</p>
    """,
    features="lxml",
)

price_selector = ClassSelector("price")
selector = NthOfSelector(price_selector, nth="2n")
selector.find_all(soup)

#### NthLastOfSelector

The `NthLastOfSelector` works similarly to `NthOfSelector` but counts elements from the end of the list. It selects the nth element matching the provided selector, starting from the last element and moving backward.

In [ ]:
import re

from bs4 import BeautifulSoup

from soupsavvy import PatternSelector
from soupsavvy.selectors.nth import NthLastOfSelector

soup = BeautifulSoup(
    """
        <span class="title">Animal Farm</span>
        <p class="price discount">Price: €1</p>
        <p>Price: $2</p>
        <span>Bestseller</span>
        <p class="price">Price: €3</p>
        <p>Price: €4</p>
        <p class="price">Price: €5</p>
    """,
    features="lxml",
)

price_selector = PatternSelector(re.compile("^price", re.IGNORECASE))
selector = NthLastOfSelector(price_selector, nth="odd")
selector.find_all(soup)

#### OnlyOfSelector

The `OnlyOfSelector` is used to select an element if it is the **only** element that matches the provided selector among its siblings. If more than one element matches the selector, none will be selected.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import ClassSelector, HasSelector
from soupsavvy.selectors.nth import OnlyOfSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Animal Farm</span>
        <p class="price">Price: $15</p>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    """,
    features="html.parser",
)
price_selector = ClassSelector("price")
selector = OnlyOfSelector(price_selector)
selector.find(soup)

#### Combining

Here's an example demonstrating how to select an element with the class `title` that is a direct child of an element containing only one child with the class `price`.

In [ ]:
from bs4 import BeautifulSoup

from soupsavvy import Anchor, ClassSelector, HasSelector
from soupsavvy.selectors.nth import OnlyOfSelector

soup = BeautifulSoup(
    """
    <div class="book">
        <span class="title">Brave New world</span>
        <p class="price">Price: $25</p>
        <p class="price">Price: $20</p>
    </div>
    <div class="book">
        <span class="title">Animal Farm</span>
    </div>
    <div class="book">
        <span class="title">Frankenstein</span>
        <p class="price">Price: $30</p>
    </div>
    """,
    features="html.parser",
)
price_selector = ClassSelector("price")
selector = HasSelector(Anchor > OnlyOfSelector(price_selector)) > ClassSelector("title")
selector.find_all(soup, recursive=False)

## API module

As an alternative way of combining selectors, `soupsavvy` offers a set of convenient API functions within the `soupsavvy.selectors.api` module, which acts as shortcuts:

- `and_` - for creating `AndSelector`
- `or_` - for creating `SelectorList`
- `is_` - for creating `SelectorList`
- `where` - for creating `SelectorList`
- `not_` - for creating `NotSelector`
- `has` - for creating `HasSelector`
- `xor` - for creating `XORSelector`

These functions could in some contexts be more clear and concise.

In [ ]:
from soupsavvy import ClassSelector, TypeSelector
from soupsavvy.api import and_

and_(ClassSelector("price"), TypeSelector("p"))

In [ ]:
from soupsavvy import ClassSelector, TypeSelector
from soupsavvy.api import has

has(ClassSelector("price"), TypeSelector("p"))

## Summary

`soupsavvy` provides a wide range of Higher Order Selectors that allow you to combine multiple selectors in various ways to create complex search criteria. Selectors were designed to be highly flexible and combinable, so that they can be customized to user specific needs without much trouble. Hopefully, this will help you to kick the ground running with this topic.

**Enjoy `soupsavvy` and leave us feedback!**  
**Happy scraping!**